In [57]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.python.data.ops.dataset_ops import CacheDataset
from tensorflow_datasets.core import DatasetInfo
print("tensorflow version: ", tf.__version__)

# PASS GAN main parameters
DATASET_NAME="rock_you"
NOISE_INPUT_SIZE = 128
BACH_SIZE = 64
EPOCHS = 199000
LAYER_DIM = 128
GRADIANT_PENALTY = 10
OUTPUT_SEQ_LENGTH = 10

# Adam Optimizer"s hyper-parameters
LEARNING_RATE = 0.0001
BETA_1 = 0.5
BETA_2 = 0.9

tensorflow version:  2.6.0


Build Networks models based on the architecture in the paper


In [58]:
# create a class Residual block based on Residual Networks definition
class ResidualBlock(tf.keras.Model):
    def __init__(self, dim):
        super().__init__()
        self.res_block = tf.keras.Sequential([
            tf.keras.layers.ReLU(True),
            tf.keras.layers.Conv1D(dim, dim, 5, padding='same'),
            tf.keras.layers.ReLU(True),
            tf.keras.layers.Conv1D(dim, dim, 5, padding='same'),
        ])

    def call(self, input_data, **kwargs):
        output = self.res_block(input_data)
        return input_data + (0.3 * output)

# create a class Generator Network
class GeneratorNetwork(tf.keras.Model):
    def __init__(self, dim, pass_length):
        self.dim = dim
        self.pass_length = pass_length

        # instantiate a Sequential Model
        self.generator_res_block_model = tf.keras.models.Sequential()

        # first linear layer
        self.first_linear_layer = tf.keras.layers.Dense(128, activation='linear',
                                                        input_shape=(dim*pass_length, ))

        # residual blocks in a sequential order
        self.generator_res_block_model.add(ResidualBlock(dim=5))
        self.generator_res_block_model.add(ResidualBlock(dim=5))
        self.generator_res_block_model.add(ResidualBlock(dim=5))
        self.generator_res_block_model.add(ResidualBlock(dim=5))
        self.generator_res_block_model.add(ResidualBlock(dim=5))

        # convolutional 1D layer
        self.conv_1d_layer = tf.keras.layers.Conv1D(64, 32, 1, padding='valid')

        # last soft max layer
        self.softmax_layer = tf.keras.layers.Softmax(axis=1)

    def call(self, input_noise, **kwargs):
        """

        :param input_noise: noise input of some sample generated passwords
        :param kwargs:
        :return: the generated passwords for an iteration
        """

        # feed first layer with noise data
        output = self.first_linear_layer(input_noise)

        # reshape the result of linear layer
        output = tf.reshape(output, (-1, 2, 128))

        # feed residual blocks by output from reshape stage
        output = self.generator_res_block_model(output)
        output = tf.reshape(output, (1, 32, 8))

        # feed resulted data to convolutional layer
        output = self.conv_1d_layer(output)

        # transpose operation on the resulted output
        output = tf.transpose(output)

        # feed softmax layer with transposed output
        output = self.softmax_layer(output)
        output = tf.reshape(output, [2, 1, 32])

        return output

# create discriminator network
class DiscriminatorNetwork(tf.keras.Model):
    def __init__(self, dim, pass_length):
        self.dim = dim
        self.pass_length = pass_length

        self.block = tf.keras.Sequential([
            ResidualBlock(dim),
            ResidualBlock(dim),
            ResidualBlock(dim),
            ResidualBlock(dim),
            ResidualBlock(dim),
        ])
        self.conv1d = tf.keras.layers.Conv1D(dim, 32, 1, padding='valid')
        self.linear = tf.keras.layers.Dense(pass_length * dim, activation='linear')

    def call(self, input_data, **kwargs):
        output = tf.transpose(input_data, [0, 2, 1])
        output = self.conv1d(output)
        output = self.block(output)
        output = tf.reshape(output, (-1, 64, 4))
        output = self.linear(output)
        return output


In [68]:
class PreprocessingPipeLine:
    def __init__(self):
        pass

    def _load_and_cache_dataset(self):
        dataset = tfds.load(name=DATASET_NAME,
                                      split='train[:75%]')
        # print("[info] ", dataset_info)
        dataset = dataset.shuffle(100000, reshuffle_each_iteration=True)

        # Cache dataset for future use
        dataset = dataset.cache()
        return dataset

    def choose_passwords_of_length_10_or_less(self, ds):
        pass

    def call(self):
        initial_ds: CacheDataset = self._load_and_cache_dataset()
        def sequence_length_filter(password: tf.Tensor):
            print(password)
            if len(password.get("password").decode("utf-8")) <= 10:
                return password
        ds = initial_ds.as_numpy_iterator()

        ds = ds.filter(sequence_length_filter)
        print("[info] dataset elements spec: ", initial_ds.element_spec)


        # ds = initial_ds.filter()
        # for data in initial_ds:
        #     print(data)
        # ds = list(filter(lambda x: len(x) <= 10, initial_ds))
        # for password in ds:
        #     print(password.get("password").decode("utf-8"))


In [69]:
PreprocessingPipeLine().call()

AttributeError: '_NumpyIterator' object has no attribute 'filter'

In [ ]:
def training(epochs=1000, batch_size=64, save_interval=500):
    # Load the rock you dataset
    pass
